In [ ]:
import os
import numpy as np
from osgeo import gdal, gdal_array
import rasterio
import geopandas as gpd
import pandas as pd
import operator
import gc

In [ ]:
ref_path = 'path to reference map of LULC'
roi_ds = gdal.Open(ref_path, gdal.GA_ReadOnly)
roi = roi_ds.GetRasterBand(1).ReadAsArray()

classes = np.unique(roi)

In [ ]:
dict = {}
for c in classes:
    dict[c] = (roi == c).sum()
sorted_x = sorted(dict.items(), key = operator.itemgetter(1))
print("Top 6 classes and pixel counts \n",sorted_x[-6:])

In [ ]:
###SUBTASK
# Select top  5 classes. Exclude 0, 255 and floating point class labels

top_classes = []

In [ ]:
n_samples = (roi > 0).sum()
print('We have {n} samples'.format(n=n_samples))

labels = np.unique(roi[roi > 0])
print('The training data include {n} classes: {classes}'.format(n=labels.size, 
                                                                classes=labels)) 
y = roi[roi > 0]

In [ ]:
#paths to bands images
images = ['path to image 1',
          'path to image 2',
          'path to image 3',
          ....,
          ....,
          'path to image n']

In [ ]:
final = pd.DataFrame()

for c in top_classes:
    temp = pd.DataFrame()
    print(c)
    for img in images:
        train_ds = gdal.Open(img, gdal.GA_ReadOnly)
        print(train_ds.RasterXSize,train_ds.RasterYSize)
        img_b1 = np.zeros((train_ds.RasterYSize, train_ds.RasterXSize, train_ds.RasterCount),
                       gdal_array.GDALTypeCodeToNumericTypeCode(train_ds.GetRasterBand(1).DataType))
        for b in range(img_b1.shape[2]):
            img_b1[:, :, b] = train_ds.GetRasterBand(b + 1).ReadAsArray()
        print(img_b1.shape)
        Xt = img_b1[roi==c, :] 
        Xt1 = pd.DataFrame(Xt)

        ###SUBTASK
        # Decide sampling number based on pixels present of each class. Default is 100000.
        Xt2 = Xt1.sample(n = 100000)

        Xt2.reset_index(drop = True, inplace = True)
        temp = pd.concat([Xt2, temp], axis = 1)
        temp["class"] = c
      
    final = pd.concat([temp, final], axis=0)
    final.reset_index(drop = True, inplace = True)
    
    gc.collect()
    final

In [ ]:
### SUBTASK
# Write a loop in the space given to change the column names to 'col_1', 'col_2', 'col_3', etc

""""""

""""""

final.head()

In [ ]:
csv_path = 'path to csv training file'
final.to_csv(csv_path, index = False)